In [1]:
"""
Created on Sat Apr 11 21:17:36 2020

@author: shivam
"""

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Dropout
import keras
from keras import optimizers

# Initialize CNN
classifier = Sequential()

# Step1: Convolution
# 32, (3, 3) means we are using 32 filters with dimention 3x3. 
# And input_shape means our input image will have 64x64 dimension and it has RGB color format i.e. 3. 
# If the input image would have been black and white then replace 3 by 1 in input_shape.
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation= 'relu'))
# Step2: Pooling
# Pool_size is the size of pooling filter i.e 2x2 in this case.
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Add one more Convolution layer
classifier.add(Convolution2D(32, (3, 3), activation= 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Add one more Convolution layer
classifier.add(Convolution2D(32, (3, 3), activation= 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Using TensorFlow backend.


In [2]:


# Step3: Flattening
classifier.add(Flatten())

# Step4: Full Connection
classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation = 'softmax', units = 1))


In [3]:

# compiling the NN
classifier.compile(optimizer = keras.optimizers.Adam(lr=.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])


In [4]:

# Pre-process our images and Fit CNN to our data. (Keras documentation have this code already)
from keras.preprocessing.image import ImageDataGenerator

# generate multiple images from 1 image by shearing, rotating etc.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

classifier.fit_generator(training_set,
#                          steps_per_epoch=8000, # no of images in our traning set
                         epochs=25,
                         validation_data=test_set
#                          validation_steps=2000
                        ) # no of images in our test set


Found 8048 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
126/126 [==============================] - 362s 3s/step - loss: 0.6886 - accuracy: 0.5337 - val_loss: 0.6630 - val_accuracy: 0.6040
Epoch 2/25
126/126 [==============================] - 359s 3s/step - loss: 0.6635 - accuracy: 0.5929 - val_loss: 0.6474 - val_accuracy: 0.6390
Epoch 3/25
126/126 [==============================] - 320s 3s/step - loss: 0.6321 - accuracy: 0.6428 - val_loss: 0.6192 - val_accuracy: 0.6760
Epoch 4/25
126/126 [==============================] - 364s 3s/step - loss: 0.5984 - accuracy: 0.6803 - val_loss: 0.4894 - val_accuracy: 0.7140
Epoch 5/25
126/126 [==============================] - 341s 3s/step - loss: 0.5785 - accuracy: 0.7022 - val_loss: 0.5114 - val_accuracy: 0.7325
Epoch 6/25
126/126 [==============================] - 264s 2s/step - loss: 0.5667 - accuracy: 0.7099 - val_loss: 0.5038 - val_accuracy: 0.7305
Epoch 7/25
126/126 [==============================] - 267s

In [5]:
# Make single predicions on random images (neighter in train nor in test data)
import numpy as np
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join

predict_dir_path='dataset/single_prediction/'
onlyfiles = [f for f in listdir(predict_dir_path) if isfile(join(predict_dir_path, f))]
print(onlyfiles)


dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(predict_dir_path+file, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = classifier.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)


['dog1.jpg', 'cat2.jpg', 'dog2.jpg', 'cat3.jpg', 'dog.jpg', 'cat_or_dog_1.jpg', 'cat.jpg', 'cat_or_dog_2.jpg']
dog1.jpg: dog
cat2.jpg: cat
dog2.jpg: dog
cat3.jpg: dog
dog.jpg: dog
cat_or_dog_1.jpg: dog
cat.jpg: dog
cat_or_dog_2.jpg: cat
Total Dogs : 6
Total Cats : 2
